In [2]:
import time
import os
import joblib # Para salvar/carregar modelos e scalers
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from optimize_model import optimize_models

JOBLIB_PATH = './joblib/'  # Caminho para salvar os modelos e scalers
METRICS_PATH = './metrics/'  # Caminho para salvar as métricas

# Certificando que a pasta existe
os.makedirs(JOBLIB_PATH, exist_ok=True)
os.makedirs(METRICS_PATH, exist_ok=True)

In [3]:
print("--- Fase 1: Carregamento e Preparação dos Dados ---")

# Carregar o dataset
cancer = load_breast_cancer()
X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = cancer.target

print(f"Formato dos dados (X): {X.shape}")
print(f"Distribuição das classes:\n{pd.Series(y).value_counts()}")
print(f"Classes: {cancer.target_names}") # ['malignant', 'benign']

# 2. Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nConjunto de Treino: {X_train.shape}")
print(f"Conjunto de Teste:  {X_test.shape}")


# 3. Padronizar os dados (Scaling) - Crucial para SVM e Redes Neurais
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Salvar o scaler para uso na API
scaler_filename = JOBLIB_PATH + 'scaler.joblib'
joblib.dump(scaler, scaler_filename)
print(f"\nScaler salvo em: {scaler_filename}")

--- Fase 1: Carregamento e Preparação dos Dados ---
Formato dos dados (X): (569, 30)
Distribuição das classes:
1    357
0    212
Name: count, dtype: int64
Classes: ['malignant' 'benign']

Conjunto de Treino: (455, 30)
Conjunto de Teste:  (114, 30)

Scaler salvo em: ./joblib/scaler.joblib


In [4]:
print("\n--- Fase 2: Treinamento e Avaliação de Modelos ---")

models = {
    "Logistic Regression": LogisticRegression(max_iter=10000, random_state=42),
    "Support Vector Machine (SVM)": SVC(random_state=42, probability=True), # probability=True pode ser mais lento, mas útil às vezes
    "Random Forest": RandomForestClassifier(random_state=42),
    "Neural Network (MLP)": MLPClassifier(max_iter=1000, random_state=42)
}

results = {}

for name, model in models.items():
  print(f"\nAvaliando modelo: {name}")

  # Medir tempo de treinamento
  start_train_time = time.time()
  model.fit(X_train_scaled, y_train)
  end_train_time = time.time()
  train_time = end_train_time - start_train_time

  # Avaliação no conjunto de teste
  y_pred = model.predict(X_test_scaled)
  accuracy = accuracy_score(y_test, y_pred)

  # Medir tempo de inferência (média de múltiplas predições individuais)
  n_inference_tests = 100
  inference_times = []
  # Pega uma amostra aleatória do conjunto de teste para simular uma requisição real
  sample_indexes = np.random.choice(X_test_scaled.shape[0], n_inference_tests, replace=True)

  for i in sample_indexes:
    sample = X_test_scaled[i].reshape(1, -1)  # Simula uma única entrada
    start_infer_time = time.perf_counter()
    model.predict(sample)
    end_infer_time = time.perf_counter()
    inference_times.append((end_infer_time - start_infer_time) * 1000)  # Convertendo para milissegundos
  
  avg_inference_time_ms = np.mean(inference_times)

  print(f"  Acurácia no Teste: {accuracy:.4f}")
  print(f"  Tempo de Treinamento: {train_time:.4f} segundos")
  print(f"  Tempo Médio de Inferência (1 amostra): {avg_inference_time_ms:.4f} ms")

  results[name] = {
      'model': model,
      'accuracy': accuracy,
      'train_time': train_time,
      'avg_inference_time_ms': avg_inference_time_ms
  }


--- Fase 2: Treinamento e Avaliação de Modelos ---

Avaliando modelo: Logistic Regression
  Acurácia no Teste: 0.9825
  Tempo de Treinamento: 0.0080 segundos
  Tempo Médio de Inferência (1 amostra): 0.1791 ms

Avaliando modelo: Support Vector Machine (SVM)
  Acurácia no Teste: 0.9825
  Tempo de Treinamento: 0.0231 segundos
  Tempo Médio de Inferência (1 amostra): 0.2581 ms

Avaliando modelo: Random Forest
  Acurácia no Teste: 0.9561
  Tempo de Treinamento: 0.2204 segundos
  Tempo Médio de Inferência (1 amostra): 6.2857 ms

Avaliando modelo: Neural Network (MLP)
  Acurácia no Teste: 0.9649
  Tempo de Treinamento: 0.4990 segundos
  Tempo Médio de Inferência (1 amostra): 0.1699 ms


In [5]:
print("\n--- Fase 3: Seleção e Otimização do Modelo ---")

# Selecionar baseado em acurácia e tempo de inferência (< 100ms)
best_model_name = None
best_accuracy = None
lowest_inference_time = float('inf')

# Prioridade: Acurácia alta e tempo de inferência baixo
potential_candidates = []
for name, res in results.items():
  if res['accuracy'] > 0.95 and res['avg_inference_time_ms'] < 100:
    potential_candidates.append((name, res['accuracy'], res['avg_inference_time_ms']))

# Ordenar por acurácia (decrescente) e tempo de inferência (crescente)
potential_candidates.sort(key=lambda x: (-x[1], x[2]))

if not potential_candidates:
  print("ALERTA: Nenhum modelo atendeu ao requisito de tempo de inferência (< 100ms). Selecionando o mais rápido.")
  potential_candidates = sorted(results.items(), key=lambda item: item[1]['avg_inference_time_ms'])
  best_model_name = potential_candidates[0][0]
else:
  best_model_name = potential_candidates[0][0] # O primeiro é o melhor (maior acurácia entre os rápidos)


print(f"\nModelo Selecionado Inicialmente: {best_model_name}")
print(f"  Acurácia: {results[best_model_name]['accuracy']:.4f}")
print(f"  Tempo Médio de Inferência: {results[best_model_name]['avg_inference_time_ms']:.4f} ms")

final_model = results[best_model_name]['model']
final_model_name = best_model_name

# Verificar se o modelo selecionado é SVM ou LogisticRegression e otimizar se for
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
if isinstance(final_model, (SVC, LogisticRegression)):
    print(f"\nOtimização do modelo {final_model_name} em andamento...")
    # Usar a função de otimização importada
    final_model_optimized, final_model_name_optimized = optimize_models(final_model, final_model_name, X_train_scaled, y_train, X_test_scaled, y_test, sample_indexes)

        # Checar se o modelo otimizado foi retornado corretamente
    if final_model_optimized is not None:
        # Avaliar o modelo otimizado
        y_pred_opt = final_model_optimized.predict(X_test_scaled)
        opt_accuracy = accuracy_score(y_test, y_pred_opt)
        
        # Medir tempo de inferência do modelo otimizado
        opt_inference_times = []
        for i in sample_indexes:
            sample = X_test_scaled[i].reshape(1, -1)
            start_infer_time = time.perf_counter()
            final_model_optimized.predict(sample)
            end_infer_time = time.perf_counter()
            opt_inference_times.append((end_infer_time - start_infer_time) * 1000)
        
        opt_avg_inference_time_ms = np.mean(opt_inference_times)
        
        # Comparar com o modelo original
        if opt_accuracy > results[best_model_name]['accuracy']:
            print(f"\nModelo otimizado selecionado: {final_model_name_optimized}")
            print(f"  Acurácia: {opt_accuracy:.4f}")
            print(f"  Tempo Médio de Inferência: {opt_avg_inference_time_ms:.4f} ms")
            final_model = final_model_optimized
            final_model_name = final_model_name_optimized
        else:
            print(f"\nModelo otimizado não melhorou a acurácia. Mantendo o modelo original.")
            print(f"  Acurácia: {results[best_model_name]['accuracy']:.4f}")
            print(f"  Tempo Médio de Inferência: {results[best_model_name]['avg_inference_time_ms']:.4f} ms")
else:
    print(f"\nO modelo {final_model_name} não é suportado para otimização. Apenas SVM e LogisticRegression são suportados.\nProsseguindo com o modelo atual.")


--- Fase 3: Seleção e Otimização do Modelo ---

Modelo Selecionado Inicialmente: Logistic Regression
  Acurácia: 0.9825
  Tempo Médio de Inferência: 0.1791 ms

Otimização do modelo Logistic Regression em andamento...

Otimizando Logistic Regression com GridSearchCV...
Tempo de Otimização: 3.21 segundos
Melhores Parâmetros: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}

Desempenho do Modelo Otimizado (Logistic Regression):
  Acurácia no Teste: 0.9737
  Tempo Médio de Inferência (1 amostra): 0.2681 ms

Modelo otimizado não melhorou a acurácia. Mantendo o modelo original.
  Acurácia: 0.9825
  Tempo Médio de Inferência: 0.1791 ms


In [6]:
print("\n--- Fase 4: Salvar Modelo Final ---")

# Salvar o modelo final treinado
model_filename = JOBLIB_PATH + 'final_cancer_classifier.joblib'
joblib.dump(final_model, model_filename)
print(f"Modelo final ('{final_model_name}') salvo em: {model_filename}")

# Guardar as métricas finais para o relatório
final_metrics = {
    "model_name": final_model_name,
    "accuracy": accuracy_score(y_test, final_model.predict(X_test_scaled)),
    "inference_time_ms": np.mean([
        (lambda s=X_test_scaled[i].reshape(1, -1): (
            t1 := time.perf_counter(),
            final_model.predict(s),
            t2 := time.perf_counter(),
            (t2 - t1) * 1000
        )[-1])()
        for i in np.random.choice(X_test_scaled.shape[0], n_inference_tests, replace=True)
    ]),
    "class_report": classification_report(y_test, final_model.predict(X_test_scaled), target_names=cancer.target_names),
    "confusion_matrix": confusion_matrix(y_test, final_model.predict(X_test_scaled)).tolist() # para fácil serialização
}

print("\nMétricas Finais do Modelo Selecionado:")
print(f"  Modelo: {final_metrics['model_name']}")
print(f"  Acurácia: {final_metrics['accuracy']:.4f}")
print(f"  Tempo Médio Inferência: {final_metrics['inference_time_ms']:.4f} ms")
print(f"  Relatório:\n{final_metrics['class_report']}")
print(f"  Matriz Confusão:\n{final_metrics['confusion_matrix']}")

# Salvar métricas para relatório (opcional)
import json
with open(METRICS_PATH + 'final_metrics.json', 'w') as f:
    json.dump(final_metrics, f, indent=4)
print("\nMétricas salvas em final_metrics.json")

print("\n--- Treinamento e Avaliação Concluídos ---")


--- Fase 4: Salvar Modelo Final ---
Modelo final ('Logistic Regression') salvo em: ./joblib/final_cancer_classifier.joblib

Métricas Finais do Modelo Selecionado:
  Modelo: Logistic Regression
  Acurácia: 0.9825
  Tempo Médio Inferência: 0.1675 ms
  Relatório:
              precision    recall  f1-score   support

   malignant       0.98      0.98      0.98        42
      benign       0.99      0.99      0.99        72

    accuracy                           0.98       114
   macro avg       0.98      0.98      0.98       114
weighted avg       0.98      0.98      0.98       114

  Matriz Confusão:
[[41, 1], [1, 71]]

Métricas salvas em final_metrics.json

--- Treinamento e Avaliação Concluídos ---
